In [35]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from pprint import pprint

In [36]:
# Dependencies
# import numpy as np
import pandas as pd
import datetime as dt

# Extract & Clean Data

### HRI Data

In [37]:
# File to Load
    # Read the ED Visits data into a Pandas DataFrame
ed_visits_df = pd.read_csv('../data/AZ-EDVisits.csv')
Hosp_df = pd.read_csv('../data/AZ_Hospitalizations.csv')
SVI_df = pd.read_csv('../data/AZ_Vulnerability.csv')
HeatDays_df = pd.read_csv('../data/AZ-NoHeatDays.csv')

In [38]:
#Pull Necessary columns and rename
ed_visits_df = ed_visits_df[['Name', 'Year', 'GeogID', 'Value']]
ed_visits_df.rename(columns={'Name': 'county', 'Year': 'year', 'GeogID': 'county_id', 'Value': 'ED_rates'}, inplace=True)
ed_visits_df.head()

,county,year,county_id,ED_rates
0,NAVAJO,2011,4017,17.72
1,NAVAJO,2011,4017,17.72
2,PIMA,2011,4019,20.12
3,PIMA,2011,4019,20.12
4,PINAL,2011,4021,37.73


In [39]:
ed_visits_df = ed_visits_df.drop_duplicates(keep='first')
ed_visits_df.head()

,county,year,county_id,ED_rates
0,NAVAJO,2011,4017,17.72
2,PIMA,2011,4019,20.12
4,PINAL,2011,4021,37.73
6,SANTA CRUZ,2011,4023,17.72
8,YAVAPAI,2011,4025,12.76


In [40]:
Hosp_df = Hosp_df[['Name', 'Year', 'GeogID', 'Value']]
Hosp_df.rename(columns={'Name': 'county', 'Year': 'year', 'GeogID': 'county_id', 'Value': 'Hosp_rates'}, inplace=True)
Hosp_df.head()

,county,year,county_id,Hosp_rates
0,GREENLEE,2011,4011,0.00
1,GREENLEE,2011,4011,0.00
2,MARICOPA,2011,4013,8.73
3,MARICOPA,2011,4013,8.73
4,MOHAVE,2011,4015,11.43


In [41]:
Hosp_df = Hosp_df.drop_duplicates(keep='first')
Hosp_df.head()

,county,year,county_id,Hosp_rates
0,GREENLEE,2011,4011,0.00
2,MARICOPA,2011,4013,8.73
4,MOHAVE,2011,4015,11.43
6,PIMA,2011,4019,4.79
8,PINAL,2012,4021,8.48


In [42]:
SVI_df = SVI_df[['Name', 'Year', 'GeogID', 'Value']]
SVI_df.rename(columns={'Name': 'county', 'Year': 'year', 'GeogID': 'county_id', 'Value': 'SVI'}, inplace=True)

SVI_df.head()

,county,year,county_id,SVI
0,APACHE,2010,4001,0.993
1,COCHISE,2010,4003,0.864
2,COCONINO,2010,4005,0.819
3,GILA,2010,4007,0.801
4,GRAHAM,2010,4009,0.984


In [43]:
HeatDays_df = HeatDays_df[['Name', 'Year', 'GeogID', 'Value']]
HeatDays_df.rename(columns={'Name': 'county', 'Year': 'year', 'GeogID': 'county_id', 'Value': 'HeatDays'}, inplace=True)

HeatDays_df.head()

,county,year,county_id,HeatDays
0,APACHE,2011,4001,7.0
1,COCHISE,2011,4003,89.0
2,COCONINO,2011,4005,2.0
3,GILA,2011,4007,81.0
4,GRAHAM,2011,4009,121.0


### Merge data to one csv

In [44]:
# Combine the data into a single dataset.  
rates_df = pd.merge(ed_visits_df, Hosp_df, left_on=['county','year'], right_on = ['county','year'])
rates_df.head()

# Remove county IDs
rates_df.drop(columns=["county_id_x", "county_id_y"])


,county,year,ED_rates,Hosp_rates
0,PIMA,2011,20.12,4.79
1,PINAL,2011,37.73,13.18
2,YUMA,2011,81.43,5.31
3,ARIZONA,2011,30.36,7.36
4,GREENLEE,2013,0.00,0.00
...,...,...,...,...
82,PINAL,2021,42.39,9.93
83,SANTA CRUZ,2021,40.66,0.00
84,YAVAPAI,2021,28.35,2.81
85,YUMA,2021,82.70,21.39


In [45]:
# Combine the data into a single dataset.  
df2 = pd.merge(SVI_df, HeatDays_df, left_on=['county','year'], right_on = ['county','year'])
df2.head()


# Remove county IDs
#rates_df.drop(columns=["county_id_x", "county_id_y"])

,county,year,county_id_x,SVI,county_id_y,HeatDays
0,APACHE,2014,4001,0.991,4001,0.0
1,COCHISE,2014,4003,0.906,4003,37.0
2,COCONINO,2014,4005,0.817,4005,0.0
3,GILA,2014,4007,0.882,4007,50.0
4,GRAHAM,2014,4009,0.952,4009,92.0


In [ ]:
# Export the cleaned DataFrame as a CSV file. 


# SQLite data to SQLAlchemy

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, distinct

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///data/HRI.sqlite")

In [ ]:
Base = automap_base() 
# reflect an existing database into a new model
Base.prepare(autoload_with=engine) 
# reflect the tables

In [ ]:
# View all of the classes that automap found
Base.classes.keys() 

In [ ]:
# Save references to each table
ED = Base.classes.ED
Hospitalizations = Base.classes.hospitalizations
HeatDays = Base.classes.hospitalizations
SVI = Base.classes.SVI

In [ ]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)

In [ ]:
session.close()